<a href="https://colab.research.google.com/github/Jason-Durr/Projects-in-Machine-Learning/blob/main/HW/Homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jason Durr

# Task 1

The machine learning problem I want to solve is predicting if the home team in an NFL Football game is going to win based on weather and betting odds.  Logistic regression makes sense for this because we are trying to predict something that only has two possible outcomes - a Win or a Loss.

# Task 2

https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data?select=spreadspoke_scores.csv

To explore the data first we proccessed it to remove missing values as trying to replace these missing values is not really an option because using an average just would not make the row have any meaningfull data.  Then we normalized the data that is numerical to have a median of 0 and an STD of 1.  The reason I chose median over mean is that I felt this would be less skewed by a few games having intense weather events and so the median would be a better place holder for depicting the "typical" weather of a game.  For the categorical variables I converted them to binary variables because games can have more than 1 weather type.  Then I looked to see if there were any large correlatiions between variables to be concerned about and as expected the only major direct relationship is between the Home team being a favorite and the variable we are trying to predict, the home team winnning. This is expected as we would expect the favorite to win more so we will need to evaluate our model based on how it does compared to just picking the favorite every time.  I also looked at the descriptive stats just to see if there were any outliers worth noting.

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"jasondurr","key":"a288e7df4fa66f235cefea7adda21090"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d tobycrabtree/nfl-scores-and-betting-data
! unzip /content/nfl-scores-and-betting-data.zip



mkdir: cannot create directory ‘/root/.kaggle’: File exists
nfl-scores-and-betting-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/nfl-scores-and-betting-data.zip
replace nfl_stadiums.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score




# Predicting Home team wins in outdoor stadiums
## Inputs

- Home favorite (Binary)
- Temperature (relative to mean)
- Humidity (relative to mean)
- Wind
- Over under relative to mean
- Rain (Binary)
- Snow (Binary)
- Fog (Binary)
- Playoff (Binary)

## Output
-  If home team wins

## Cases to Remove
- Week 17
- Neutral Stadiums


In [ ]:
data = pd.read_csv('spreadspoke_scores.csv')
teams = pd.read_csv('nfl_teams.csv')

In [ ]:
data.columns

Index(['schedule_date', 'schedule_season', 'schedule_week', 'schedule_playoff',
       'team_home', 'score_home', 'score_away', 'team_away',
       'team_favorite_id', 'spread_favorite', 'over_under_line', 'stadium',
       'stadium_neutral', 'weather_temperature', 'weather_wind_mph',
       'weather_humidity', 'weather_detail'],
      dtype='object')

In [ ]:
df = data.dropna(subset=['team_favorite_id','weather_temperature'])
df = df[df['stadium_neutral'] == False]
df = df[df['schedule_week'] != 17]
df.weather_detail = df.weather_detail.fillna('Clear')
df.over_under_line = df.over_under_line.replace(' ',np.NaN)
df.weather_detail.unique()

array(['Clear', 'DOME', 'Rain', 'Rain | Fog', 'Fog', 'Snow',
       'DOME (Open Roof)', 'Snow | Fog', 'Snow | Freezing Rain'],
      dtype=object)

In [ ]:
X = pd.DataFrame()
X["Home_Favorite"] = np.where(df['team_home'].map(teams.set_index('team_name')['team_id']) == df['team_favorite_id'], 1, 0)
X["Temperature"] = (df.weather_temperature.to_numpy() - data.weather_temperature.median())/df.weather_temperature.apply(lambda x: float(x)).std()
X["Humidity"] = (df.weather_humidity.apply(lambda x: float(x)).to_numpy() - df.weather_humidity.apply(lambda x: float(x)).median())/df.weather_humidity.apply(lambda x: float(x)).std()
X["Wind"] = (df.weather_wind_mph.to_numpy() - data.weather_wind_mph.median())/df.weather_wind_mph.apply(lambda x: float(x)).std()
X["Over_Under"] = (df.over_under_line.apply(lambda x: float(x)).to_numpy() - df.over_under_line.apply(lambda x: float(x)).median())/df.over_under_line.apply(lambda x: float(x)).std()
X['Rain'] = df.weather_detail.apply(lambda x: int('Rain' in x)).to_numpy()
X['Snow'] = df.weather_detail.apply(lambda x: int('Snow' in x)).to_numpy()
X['Fog'] = df.weather_detail.apply(lambda x: int('Fog' in x)).to_numpy()
X['Playoff'] = df.schedule_playoff.apply(lambda x: int(x)).to_numpy()
X["Home_Win"] = np.where(df.score_home > df.score_away,1,0)

In [ ]:
X = X.dropna()

In [ ]:
X

,Home_Favorite,Temperature,Humidity,Wind,Over_Under,Rain,Snow,Fog,Playoff,Home_Win
9,1,1.165432,1.184926,0.175000,-2.555330,0,0,0,0,1
10,0,0.841701,0.374187,1.225003,-0.638833,0,0,0,0,0
11,1,1.100685,0.000000,0.525001,-2.342386,0,0,0,0,1
12,1,0.517970,-1.870936,-0.350001,-2.235914,0,0,0,0,1
13,1,0.971193,0.187094,0.000000,-1.064721,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
8242,1,-1.294924,1.559114,-0.175000,0.532360,1,0,1,0,1
8243,1,0.776954,-3.367686,-1.050002,0.638833,0,0,0,0,1
8244,1,-1.100685,1.559114,-0.525001,-0.212944,0,0,0,0,1
8245,1,-0.517970,-0.436552,-0.700002,0.532360,0,0,0,0,1


In [ ]:
  X.corr()

,Home_Favorite,Temperature,Humidity,Wind,Over_Under,Rain,Snow,Fog,Playoff,Home_Win
Home_Favorite,1.000000,-0.015593,0.008292,0.033777,-0.017991,-0.003455,0.016447,0.003594,-0.018301,0.282557
Temperature,-0.015593,1.000000,-0.023081,-0.193344,0.086904,0.013382,-0.086626,-0.016689,0.006008,-0.039486
Humidity,0.008292,-0.023081,1.000000,0.036437,-0.067725,0.083314,0.020739,0.089699,0.009018,-0.010055
Wind,0.033777,-0.193344,0.036437,1.000000,-0.117398,-0.060438,-0.002021,-0.048340,-0.002949,0.030644
Over_Under,-0.017991,0.086904,-0.067725,-0.117398,1.000000,0.028735,0.022901,0.046136,0.041078,-0.011113
Rain,-0.003455,0.013382,0.083314,-0.060438,0.028735,1.000000,0.030882,0.401846,-0.001262,0.021767
Snow,0.016447,-0.086626,0.020739,-0.002021,0.022901,0.030882,1.000000,0.166155,-0.000576,-0.011541
Fog,0.003594,-0.016689,0.089699,-0.048340,0.046136,0.401846,0.166155,1.000000,-0.001049,0.007026
Playoff,-0.018301,0.006008,0.009018,-0.002949,0.041078,-0.001262,-0.000576,-0.001049,1.000000,0.010608
Home_Win,0.282557,-0.039486,-0.010055,0.030644,-0.011113,0.021767,-0.011541,0.007026,0.010608,1.000000


In [ ]:
X.describe()

,Home_Favorite,Temperature,Humidity,Wind,Over_Under,Rain,Snow,Fog,Playoff,Home_Win
count,6272.000000,6272.000000,6272.000000,6272.000000,6272.000000,6272.000000,6272.000000,6272.000000,6272.000000,6272.000000
mean,0.677455,-0.350127,-0.090484,0.357785,-0.273205,0.009885,0.002073,0.006856,0.000159,0.586256
std,0.467487,1.008089,1.000097,0.782896,0.902322,0.098940,0.045483,0.082522,0.012627,0.492543
min,0.000000,-4.143757,-3.991331,-1.400003,-2.981218,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,-1.035939,-0.748375,-0.175000,-0.958249,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,-0.258985,0.000000,0.175000,-0.319416,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.388477,0.623645,0.700002,0.319416,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,2.201371,1.995665,4.200009,4.471828,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
Y = np.matrix(X['Home_Win']).T
x = X.drop('Home_Win',axis=1).to_numpy()

In [ ]:
sample = np.random.choice(x.shape[0],size=x.shape[0]//5)
x_train = x[~sample,:]
x_test = x[sample,:]
y_train = Y[~sample]
y_test = Y[sample]

In [ ]:
x

array([[ 1.        ,  1.1654317 ,  1.18492639, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.84170067,  0.37418728, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.10068549,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        , -1.10068549,  1.55911367, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -0.51796964, -0.43655183, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.12949241,  0.62364547, ...,  0.        ,
         0.        ,  1.        ]])

# Task 3 Implement Logistic Regression

In [ ]:
def sigmoid(X):
  return (1/(1+np.exp(-X)))


In [ ]:
def logisticRegression(X,Y,batchSize=-1,epochs=1000,eta=.001):
  w = np.matrix(np.random.random((1,X.shape[1])))
  b = np.random.random((1))
  for i in range(epochs):
    if batchSize > 0:
      sample = np.random.choice(X.shape[0],size=batchSize)
      x = X[sample,:]
      y = Y[sample]
    else:
      x = X
      y = Y
    n= x.shape[0]
    A = sigmoid(np.dot(w,x.T) + b).T

    # cost=-1/m * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))
    dw = np.dot(x.T, (A-y))/n
    db= np.sum(A-y)/n
    w = w - eta*dw.T
    b= b - eta*db
    if i % 10000 == 0:
      print(np.sum(np.square(Y-sigmoid(np.dot(w,X.T) + b).T)))
  return w,b

### Batch Gradient Descent with learning rate .01

In [ ]:
w,b = logisticRegression(x_train,y_train,epochs=100000,eta=.01)

print("Batch Accuracy:",accuracy_score(y_test,np.where(sigmoid(np.dot(w,x_test.T) + b).T>.5,1,0)))

320.43189174481245
283.3788122802628
283.1454587537944
282.96853417640887
282.8290142853897
282.7198961459402
282.6349212956154
282.5688185905369
282.5173194799447
282.4770540419793
Batch Accuracy: 0.671451355661882


### Stochastic Gradient Descent with learning rate .01

In [ ]:
w,b = logisticRegression(x_train[:,0:],y_train,batchSize=1,epochs=100000,eta=.01)

print("Stochastic Accuracy: ",accuracy_score(y_test,np.where(sigmoid(np.dot(w,x_test[:,0:].T) + b).T>.5,1,0)))

375.25820355724727
285.02193395548284
285.3274889275977
284.4622442652077
283.65639735083784
286.08127771760314
284.8476180433594
284.5291236142985
282.85180875931695
282.7029909472927
Stochastic Accuracy:  0.6690590111642744


### For Comparison The Accuracy of just picking the favorite

In [ ]:
print("Always Pick the Favorite Accuracy:",accuracy_score(y_test,x_test[:,0]))

Always Pick the Favorite Accuracy: 0.6690590111642744


# Task 4 Implement optimization Algorithms

In [ ]:
def logisticRegressionMomentum(X,Y,batchSize=-1,epochs=1000,eta=.001,gamma=.9):
  w = np.matrix(np.random.random((1,X.shape[1])))
  b = np.random.random((1))
  vw_1 = None
  vb_1 = None
  for i in range(epochs):
    if batchSize > 0:
      sample = np.random.choice(X.shape[0],size=batchSize)
      x = X[sample,:]
      y = Y[sample]
    else:
      x = X
      y = Y
    n= x.shape[0]
    A = sigmoid(np.dot(w,x.T) + b).T

    # cost=-1/m * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))
    
    dw = np.dot(x.T, (A-y))/n
    db= np.sum(A-y)/n
    if i > 1:
      vw = gamma*vw_1 + eta*dw
      vb = gamma*vb_1 + eta*db
    else:
      vw = eta * dw
      vb = eta*db
    w = w - vw.T
    b= b - vb
    vw_1 = vw
    vb_1 = vb
    if i % 10000 == 0:
      print(np.sum(np.square(Y-sigmoid(np.dot(w,X.T) + b).T)))
  return w,b

In [ ]:
w,b = logisticRegressionMomentum(x_train,y_train,epochs=100000,eta=.01)

print("Batch Accuracy with Momentum:",accuracy_score(y_test,np.where(sigmoid(np.dot(w,x_test.T) + b).T>.5,1,0)))

392.3316239259094
282.33792478249086
282.28019826659397
282.2634752275634
282.250765271271
282.2404689854434
282.2322969251991
282.2258579514305
282.2207647007034
282.21669875492057
Batch Accuracy with Momentum: 0.6706539074960127


In [ ]:
def logisticRegressionNesterov(X,Y,batchSize=-1,epochs=1000,eta=.001,gamma=.9):
  w = np.matrix(np.random.random((1,X.shape[1])))
  b = np.random.random((1))
  vw_1 = None
  vb_1 = None
  for i in range(epochs):
    if batchSize > 0:
      sample = np.random.choice(X.shape[0],size=batchSize)
      x = X[sample,:]
      y = Y[sample]
    else:
      x = X
      y = Y
    n= x.shape[0]
    if i > 1:
      w = w - vw_1.T
      b= b - vb_1
    A = sigmoid(np.dot(w,x.T) + b).T

    # cost=-1/m * np.sum(Y * np.log(A) + (1-Y) * (np.log(1-A)))
    
    dw = np.dot(x.T, (A-y))/n
    db= np.sum(A-y)/n
    if i > 1:
      vw = gamma*vw_1 + eta*dw
      vb = gamma*vb_1 + eta*db
    else:
      vw = eta * dw
      vb = eta*db
    w = w - vw.T
    b= b - vb
    vw_1 = vw
    vb_1 = vb
    if i % 10000 == 0:
      print(np.sum(np.square(Y-sigmoid(np.dot(w,X.T) + b).T)))
  return w,b

In [ ]:
w,b = logisticRegressionNesterov(x_train,y_train,epochs=100000,eta=.01)

print("Batch Accuracy with Nesterov:",accuracy_score(y_test,np.where(sigmoid(np.dot(w,x_test.T) + b).T>.5,1,0)))

417.24551236772334
282.28151548979275
282.2510612298516
282.23269408268624
282.22104049620197
282.2135994091818
282.2086494459289
282.20521431608694
282.20274044708447
282.2009018349487
Batch Accuracy with Nesterov: 0.6706539074960127


### Task 4 results

These optimiztion algorithms achieved the same accuracy score on the sample however Nesterov was able to achieve it faster than the simple momentum algorithm as seen by looking at the printout of the mean square error.  Both of these were able to do this faster than the regular batch algorithm and the stochastic gradient descent.  It makes sense to use these algorithms in order to find the minimum faster, this time savings can then be used to better tune the hyper parameters to find a more optimal solution.  The downside however is that the test accuracy is slightly worse than that of the regular batch algorithm.